<a href="https://colab.research.google.com/github/lishav123/MLS-track-notes-/blob/main/Introduction_to_DeepLearning_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.tensor([1, 2, 3, 4, 5])

tensor([1, 2, 3, 4, 5])

In [11]:
from sklearn.datasets import load_iris

X = torch.tensor(load_iris().data).float()
y = torch.tensor(load_iris().target).long()

model = torch.nn.Sequential(
    torch.nn.Linear(4, 3),
    torch.nn.ReLU(),
    torch.nn.Linear(3, 4),
    torch.nn.ReLU(),
    torch.nn.Linear(4, 3),
    torch.nn.Softmax(dim=1)
)

optim = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(1000):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    optim.zero_grad()
    loss.backward()
    optim.step()

loss = loss_fn(model(X), y)
print(loss)

tensor(0.6446, grad_fn=<NllLossBackward0>)


In [12]:
from sklearn.metrics import accuracy_score

accuracy_score(y, torch.argmax(model(X), dim=1))

0.98